# NSRR CCSHS Features Extraction

Unfortunately, for this dataset we do not have metric of signal quality and scoring reliability.

**WARNING:** 

- All EEG, EOG and EMG are referenced to Fpz.
- Sampling rate is 128 Hz with a highpass at 0.15 Hz.

https://sleepdata.org/datasets/ccshs/pages/montage-and-sampling-rate-information.md

In [ ]:
import os
import yasa
import platform
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from mne.io import read_raw_edf

# Define paths to the CCSHS dataset
if platform.system() == 'Darwin':
    root_dir = '/Volumes/NSRR/ccshs/'
else:
    root_dir = 'G:/ccshs/'

eeg_dir = root_dir + 'polysomnography/edfs/'
hypno_dir = root_dir + 'polysomnography/annotations-events-profusion/'
desc_dir = root_dir + 'datasets/ccshs-trec-dataset-0.3.0.csv'
df_subj = pd.read_csv(desc_dir, usecols=['nsrrid', 'age', 'ethnicity', 'male', 'bmi', 'race3', 'ahi_a0h3'])

# Rename for ease-of-use
df_subj['race3'].replace({1: 'caucasian', 2: 'african', 3: 'other'}, inplace=True)
df_subj.loc[df_subj['ethnicity'] == 1, 'race3'] = 'hispanic'
df_subj.drop(columns=['ethnicity'], inplace=True)
df_subj.rename(columns={'race3': 'ethnicity', 'nsrrid': 'subj', 'ahi_a0h3': 'ahi'}, inplace=True)

# Remove subject for which we don't have the data
df_subj = df_subj[~df_subj['subj'].isin([1800639, 1800842])]

# Convert to str
df_subj['subj'] = df_subj['subj'].astype(str)
df_subj.set_index('subj', inplace=True)

# Export demographics to CSV file
df_subj['dataset'] = 'CCSHS'
df_subj.to_csv("/Volumes/JAWA/SHERLOCK/demo/demo_nsrr_ccshs.csv")

# Keep only a random subset of 420 subjects (leave ~100 for testing)
df_subj = df_subj.sample(n=420, replace=False, random_state=42)

print(df_subj.shape[0], 'subjects remaining')
df_subj.head(10)

In [ ]:
df = []
include = ['C4', 'LOC', 'EMG1']
sf = 100

for sub in tqdm(df_subj.index):
    eeg_file = eeg_dir + 'ccshs-trec-' + sub + '.edf'
    hypno_file = hypno_dir + 'ccshs-trec-' + sub + '-profusion.xml'

    # LOAD EEG DATA
    try:
        # To deal with MNE loading error i_sub = 126
        raw = read_raw_edf(eeg_file, preload=False, verbose=0)
        raw = read_raw_edf(eeg_file, preload=True, 
                           exclude=np.setdiff1d(raw.info['ch_names'], include), 
                           verbose=0)
    except:
        continue
        
    # Resample and low-pass filter 
    raw.resample(sf, npad="auto")
    
    # LOAD HYPNOGRAM
    hypno, sf_hyp = yasa.load_profusion_hypno(hypno_file)
    # We keep up to 15 minutes before / after sleep
    start_to_firstsleep_min = np.nonzero(hypno)[0][0] / 2
    lastsleep = np.nonzero(hypno)[0][-1]
    lastsleep_to_end_min = (len(hypno) - lastsleep) / 2
    tmin, tmax = 0, None  # must be in seconds
    if start_to_firstsleep_min > 15:
        tmin = (start_to_firstsleep_min - 15) * 60
    if lastsleep_to_end_min > 15:
        tmax = lastsleep * 30 + 15 * 60
    # Crop!
    raw.crop(tmin, tmax)
    if tmax is None:
        hypno = hypno[int(tmin / 60 * 2):]
    else:
        hypno = hypno[int(tmin / 60 * 2):int(tmax / 60 * 2)]
    # Hypno and data have the same number of epochs
    n_epochs = hypno.shape[0]
    if n_epochs != np.floor(raw.n_times / sf / 30):
        print("- Hypno and data size do not match.")
        continue
    
    # Convert hypnogram to str
    df_hypno = pd.Series(hypno)
    df_hypno.replace({0: 'W', 1: 'N1', 2: 'N2', 3: 'N3', 4: 'R'}, inplace=True)
    stage_min = df_hypno.value_counts(sort=False) / 2

    # INCLUSION CRITERIA
    # Hypnogram must include all stages
    if np.unique(hypno).tolist() != [0, 1, 2, 3, 4]:
        print("- Not all stages are present.")
        continue
    # If the duration is not between 4 to 12 hours, skip subject
    if not(4 < n_epochs / 120 < 12):
        print("- Recording too short/long.")
        continue
    # Requires at least 5 min of each stage
    # if (stage_min < 5).any():
    #    print("- Not 5 min of each stage.")
    #    continue
       
    # EXTRACT FEATURES
    metadata = dict(age=df_subj.loc[sub, 'age'], male=df_subj.loc[sub, 'male'])
    sls = yasa.SleepStaging(raw, eeg_name=include[0], eog_name=include[1], 
                            emg_name=include[2], metadata=metadata)

    features = sls.get_features().reset_index()
    features['subj'] = sub
    features.set_index(['subj', 'epoch'], inplace=True)
    
    # Add hypnogram
    features['stage'] = df_hypno.to_numpy()
    df.append(features)

df = pd.concat(df)

In [ ]:
# Add dataset
df['dataset'] = 'ccshs'

# Convert to category
df['dataset'] = df['dataset'].astype('category')
df['stage'] = df['stage'].astype('category')

In [ ]:
# Show %stage
df['stage'].value_counts(normalize=True, sort=True)

In [ ]:
# Median value of the EEG IQR per stage
df.groupby('stage')['eeg_iqr'].median()

In [ ]:
# Number of unique nights in dataset
df.index.get_level_values(0).nunique()

In [ ]:
# Export feature file
if platform.system() == 'Darwin':
    df.to_parquet("/Volumes/JAWA/SHERLOCK/features/features_nsrr_ccshs.parquet")
else:
    df.to_parquet("H:/SHERLOCK/features/features_nsrr_ccshs.parquet")